## <font color='blue'>SME0104 - Cálculo Numérico</font>

### Primeiro Trabalho em Grupo 
#### Comparação de Métodos na Solução do Laplaciano em Grafos para propagação de informação

**Luis Gustavo Nonato**

------

Considere os arquivos `manh.el` e `manh.xy` que fornecem as arestas e as coordenadas dos vértices do grafo de ruas da ilha de Manhattan, NY (arquivos disponíveis para download no Google Drive).

O grafo de ruas possui diversas componentes conexas, considerando somente a maior componente conexa, você deve realizar as seguintes tarefas:
1. Selecione alguns vértices do grafo $v_{i_1},v_{i_2},\ldots,v_{i_k},\, k<<n$ ($n$ é o número de vértices na maior componente do grafo e $k$ é um número bem menor que $n$, $k=10$ por exemplo) e atribua valores distindos $c_{i_1},c_{i_2},\ldots,c_{i_k}$ a cada um dos vértices selecionados (por exemplo valores no intervalo (0,10]);

---


2. Construa a matriz Laplaciana $L$ do grafo de ruas;
3. Construa a matriz de penalidades $P$, sendo $P$ é uma matriz diagonal onde a entrada $P_{jj}=\alpha$ se $j$ corresponde ao índice de algum dos vértices escolhidos no item 1 acima ($\alpha=1.0e7$ por exemplo), sendo $P_{ii}=0$ caso contrário.
4. Construa um vetor $b$ da seguinte forma:
$$
b_{j} = \left\{\begin{matrix} c_{i_s} & \mbox{ se } j = i_s \\ 0 & \mbox{ caso contrário}\end{matrix}\right.
$$
5. Compare o tempo de solução do sistema
$$
(L+P)x = Px
$$
para os métodos:
    - Decomposição LU
    - Cholesky
    - Jacobi e Gaus-Seidel
    - Gradientes Conjugados
    
6. Refaça as tarefas com representação por matriz esparsa e matrizes cheias, comparando os resultados.

Alunos: 
- Kauê Hunnicutt Bazilli - 11212226
- Matheus
- Pedro Henrique dias Junqueira de Souza - 11294312

In [23]:
import sys
%pip install --prefix {sys.prefix} numpy
%pip install --prefix {sys.prefix} scipy
%pip install --prefix {sys.prefix} networkx

import numpy as np
import scipy as sci
import networkx as nx

import scipy.linalg

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


In [10]:
# Construção do grafo em memória com base nos arquivos
G = nx.Graph()

#Nós
with open("./manh.xy","r") as file:
    lines = file.readlines()
    for i, line in enumerate(lines):
        [x, y] = map(float, line.split("\t")) #Lê como floats
        G.add_node(i, data=(x, y))

#Arestas
with open("./manh.el","r") as file:
    lines = file.readlines()
    for line in lines:
        [x, y] = map(int, line.split("\t")) #Lê como ints
        G.add_edge(x,y)
        
# Obtendo maior componente conexa
LCC_generator = max(nx.connected_components(G), key=len)
G = G.subgraph(LCC_generator).copy()

# Desse ponto em diante, G é a maior componente conexa

In [11]:
# 1) Selecione alguns vértices do grafo e atribua valores distintos a cada um dos vértices selecionados

# Uma lista dos índices e outra lista com os seus respectivos valores
c_indexes = np.random.randint(0, len(G), 100)
c_values = np.random.uniform(0, 10, len(c_indexes))

In [12]:
# 2) Construa a matriz Laplaciana L do grafo de ruas

# Obtendo a matriz laplaciana
L = np.array(nx.laplacian_matrix(G).todense())


<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [13]:
# 3) Construa a matriz de penalidades P

P_diag =  [1e7 if i in c_indexes else 0 for i in range(len(G))]
P = np.diag(P_diag)

In [14]:
# 4) Construa o vetor b 
# Px = b
# b é um vetor com ci nos índices i escolhidos na 1) e 0 em todo o resto.
b = np.zeros(len(G))
for i,ci in enumerate(c_indexes):
    b[ci] = c_values[i]

In [38]:
# 4.1) Testando convergência dos métodos
#   - Decomposição LU
#   - Cholesky
#   - Jacobi e Gaus-Seidel
#   - Gradientes Conjugados

# Matriz A do sistema
A = np.add(L, P)

# Teste da existência da decomposição LU:
# Verificando se os menores principais tem determinantes não-nulos
# Apesar de ser uma condição suficiente, esse teste é bastante lento
# dado o número grande de operações necessárias para calcular os determinantes.
def firstMinorDeterminantsNonNull(m):
    # Determinantes dos menores principais
    determinants = []
    for i in range(1,len(m)+1):
        determinants.append(sci.linalg.det(m[0:i,0:i]))
    
    # Verificando se são não nulos
    for i in range(len(determinants)):
        if determinants[i] == 0: return False
    return True

lu_possible = True
print(f"Decomposição LU é possível: {lu_possible}")

# Teste para a existência da decomposição de Cholesky:
# Verificar se a matriz é simétrica e tem os menores principais não-nulos
# (Mesmo critério da decomposição LU)
def isSymmetric(m):
    #Compara matriz com sua transposta
    return np.allclose(m, m.T) 

cholesky_possible = isSymmetric(L) and lu_possible
print(f"Decomposição de Cholesky é possível: {cholesky_possible}")

# Teste de convergência do método de Jacobi:
# Matriz estritamente diagonal dominante (critério das linhas)
# Critério das linhas
def row_criterion(m):
    #Para cada linha, calcula a soma dos módulos de todo elemento da linha,
    # e verifica se essa soma é maior ou igual ao elemento diagonal. Se for,
    # interrompe a verificação (o critério falhou)
    for i in range(len(m)):
        line_sum = 0
        for j in range(len(m[i])):
            if(i != j):
                line_sum += abs(m[i][j])
        
        if(line_sum >= abs(m[i][i])):
            return False

    return True
        
jacobi_possible = row_criterion(A)
print(f"Método de Jacobi é possível: {jacobi_possible}")

# TODO: testes de convergência para Gauss-seidel e gradientes conjugados, rever cholesky

Decomposição LU é possível: True
Decomposição de Cholesky é possível: True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
1.0 False
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 True
0.0 Tr

In [16]:
# TODO: 5) Compare o tempo de solução do sistema (L + P)x = Px = b para os métodos:
#   - Decomposição LU
#   - Cholesky
#   - Jacobi e Gaus-Seidel
#   - Gradientes Conjugados
# NOTE: Pode usar métodos embutidos do numpy e etc...
#
# Lx = 0
# Px = b
# 
# Nem P nem L são invertíveis, mas (L + P) é, então resolvemos a soma dos sistemas:
#
# (L + P)x = Px = b

In [17]:
# Decomposição LU
# TODO